# HO02: Text Clustering
*Vencimento* 16 mai por 7:00 *Pontos* 5

## Problema
Clusterizar o conjunto de dados 20 News Group Dataset, vetorizando o dataset utilizando TF-IDF e Word2Vec, utilizando cada uma das abordagens abaixo:

- K-Means (K=4)
- Spectral Clustering (K=6)
- Gaussian Mixture
- Agglomerative Clustering
- DBSCAN
- HDBSCAN

Para cada cluster, mostrar os top-10 documentos e os top-20 tokens que representam o cluster.

**usar biblioteca python para baixar esse dataset**
<br>
https://builtin.com/data-science/tsne-python
https://github.com/VeereshElango/tsne-visualizations/blob/master/notebooks/t-SNE%20visualization%20of%2020%20News%20group%20dataset.ipynb

Clustering são algoritmos não supervisionados, explotarory data mining, analise prévia
Clustering não tem rótulos.

### 1° passo: Carregar as informações

In [1]:
# Baixar o dataset do 20 News Group 
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
import hdbscan
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

import nltk
from gensim.models import Word2Vec

news = fetch_20newsgroups(subset='train')
stop_words = stopwords.words('english')

In [2]:
list(news.target_names)

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [3]:
categorias = ['comp.windows.x',  'sci.electronics']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categorias)
print(f'Exibir primeiras 10 linhas de {len(newsgroups_train.data)} do dataset:')
newsgroups_train.data[:10]

Exibir primeiras 10 linhas de 1184 do dataset:


['From: cfb@fc.hp.com (Charlie Brett)\nSubject: Re: Los Angeles Freeway traffic reports\nNntp-Posting-Host: hpfcmgw.fc.hp.com\nOrganization: Hewlett-Packard Fort Collins Site\nX-Newsreader: TIN [version 1.1 PL8.5]\nLines: 21\n\n: While driving through the middle of nowhere, I picked up KNBR, AM 1070,\n: a clear-channel station based in Los Angeles. They had an ad \n: claiming that they were able to get traffic flow information from \n: all of the thousands of traffic sensors that CalTrans has placed\n: under the pavement. Does CalTrans sell this info? Does KNBR have\n: an exclusive? What\'s the deal?\n\n: ==Doug "Former L.A. commuter" Claar\n\nYou were right the second time, it is KNX. Believe it or not, I also\nlisten to KNX in the evenings here in Colorado! It\'s kind of fun driving\nthrough the country listening to traffic jams on the 405. Back to your\noriginal question. Yes, there are sensors just past every on-ramp and\noff-ramp on the freeways. They\'re the same sensors used at 

### Preparação dos dados: Normalização e Vetorização
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [60]:
def normalizar(texto):
    texto = texto.replace('\n', ' ') # Remove quebra de linha
    texto = texto.lower() # Converte para minúsculo
    texto = re.sub(r'[^a-zA-Z0-9@\'\s]', '', texto) # Remove caracteres especiais
    texto = ' '.join([palavra for palavra in texto.split() if palavra not in stop_words]) #remove stopwords
    return texto

documentos = [normalizar(texto) for texto in newsgroups_train.data]

# Extrai as features do dataset usando TF-IDF
tfidf_vectorizer = TfidfVectorizer(min_df=0.01) #ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature.
tfidf_vectors = tfidf_vectorizer.fit_transform(documentos)
df_tfidf = pd.DataFrame(tfidf_vectors.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
df_tfidf

,10,100,11,12,120,120vac,121,122,13,14,...,xterminal,xview,xwindows,year,years,yes,yet,york,young,zero
0,0.000000,0.000000,0.066876,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.087431,0.0,0.0,0.0,0.0
1,0.031507,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.039397,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.379575,0.168979,0.055138,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.058468,0.000000,0.0,0.0,0.0,0.0,0.063011,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1179,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1180,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1181,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1182,0.000000,0.023417,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.044382,0.0,0.0,0.0,0.0


In [ ]:
# Extrai as features do dataset usando Word2Vec

documentos_lista = [noticia.split() for noticia in documentos]
# treinamento do modelo Word2Vec
w2v_model = Word2Vec(sentences=documentos_lista, min_count=1)
# representação das notícias em vetores
w2v_vectors = [w2v_model.wv[noticia.lower().split()] for noticia in documentos]

print(list(w2v_model.wv.key_to_index))      # Obtem a lista de palavras processadas )
print(w2v_vectors)

In [65]:
w2v_vectors[2][0]

array([-0.01652006,  0.01419332,  0.0122819 , -0.00180893,  0.00512044,
       -0.0073957 ,  0.00846095,  0.02881219,  0.00506732, -0.00121433,
       -0.00352102, -0.02235141, -0.0006537 ,  0.00258701, -0.00673456,
       -0.01207881,  0.00604758, -0.01065492, -0.0032476 , -0.02148976,
       -0.00228115,  0.00389036,  0.0054407 , -0.00529856, -0.00538413,
        0.0029672 , -0.00731137, -0.00091297,  0.00258468,  0.00823053,
        0.00638312, -0.00450728, -0.00044623, -0.01369725,  0.00610087,
        0.02663051, -0.00034929, -0.01270448, -0.01409732, -0.00730491,
        0.00939848, -0.01364678,  0.00139648,  0.01266195,  0.01711779,
        0.00488347,  0.00109441, -0.00780397,  0.01383318,  0.01097579,
        0.01379389, -0.01585384, -0.00184106,  0.00065383, -0.02445474,
        0.01664529,  0.01199625,  0.00077537, -0.0004598 , -0.00924737,
        0.00976781,  0.00913684, -0.00070106,  0.00333541, -0.01434297,
        0.01379577,  0.00949759,  0.01303173, -0.0080032 ,  0.01

In [67]:
i=2
print(' '.join(x for (x,y) in w2v_model.wv.most_similar(w2v_vectors[i])))
print(documentos[i])

from sklearn.decomposition import PCA


pca = PCA(n_components=1)
vetor_unidimensional = pca.fit_transform(w2v_vectors[i].T)
vetor_unidimensional.shape


motorola canada access 10 uk 30 division 408 electrical 20
grape@suned1nswsesnavymil mike grapevine subject subscription organization internet lines 7 xpert@expolcsmitedu please subscribe mailing list michael grapevine one days usn swc one days code 4g33 pow grape@suned1nswsesnavymil gonna happen bus 805 9827791 even realize


(100, 1)

In [59]:
from gensim.models import Word2Vec

# List of documents
documents = [
    ['O', 'rato', 'roeu', 'a', 'roupa', 'do', 'rei', 'de', 'Roma'],
    ['O', 'rei', 'de', 'Roma', 'desceu', 'o', 'morro'],
    ['Roma', 'não', 'se', 'fez', 'em', 'um', 'dia']
]

# Create the Word2Vec model and build the vocabulary
modelo = Word2Vec(sentences=documents, min_count=1)

# Print the words in the vocabulary
print(modelo.wv.index_to_key)




['Roma', 'de', 'rei', 'O', 'se', 'não', 'morro', 'o', 'desceu', 'um', 'fez', 'em', 'do', 'roupa', 'a', 'roeu', 'rato', 'dia']


[['O', 'rato', 'roeu', 'a', 'roupa', 'do', 'rei', 'de', 'Roma'],
 ['O', 'rei', 'de', 'Roma', 'desceu', 'o', 'morro'],
 ['Roma', 'não', 'se', 'fez', 'em', 'um', 'dia']]

In [ ]:
#Como reduzir a tridimensionalidade de Word2Vec para 2D??

from sklearn.decomposition import PCA

pca = PCA(n_components=1)
w2v_vectors_reduzida = pca.fit_transform(w2v_vectors[0])

w2v_vectors_reduzida = []
for doc in w2v_vectors:
    w2v_vectors_reduzida.append(np.mean(doc, axis=0))
w2v_vectors_reduzida

# Desenvolvento os algoritmos de Clusterização

In [80]:
def kmeans_cluster(vetor, termos, k=4):
    kmeans = KMeans(n_clusters=k, random_state=0, n_init=10).fit(vetor)
    
    top_docs = []        
    top_tokens = []
    order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
    for i in range(kmeans.n_clusters):
        # Obtém os índices dos top 10 documentos de cada cluster
        cluster_docs = np.where(kmeans.labels_ == i)[0]
        cluster_scores = vetor[cluster_docs].sum(axis=1)
        top_docs_idx = np.argsort(cluster_scores, axis=0)[::-1][:10]
        top_docs.append(cluster_docs[top_docs_idx])

        # Obtém os termos mais importantes de cada cluster# Obtém os 20 tokens mais representativos de cada cluster
        cluster_tokens = [termos[ind] for ind in order_centroids[i, :20]]
        top_tokens.append(cluster_tokens)   
    
    # Imprime os top  documentos de cada cluster
    for i, docs in enumerate(top_docs):
        print(f"Cluster {i}:\nTop 20 tokens representativos: \n\t{', '.join(top_tokens[i])}\nTop 10 docs:")
        
        for doc_idx in docs:
            print(f"\t{documentos[doc_idx[0]]}")
        print("="*80 + "\n")

kmeans_cluster(tfidf_vectors, tfidf_vectorizer.get_feature_names_out())


Cluster 0:
Top 20 tokens representativos: 
	one, would, use, writes, good, ground, power, article, radar, audio, lines, like, work, subject, current, radio, circuit, chip, organization, amp
Top 10 docs:
	jeh@cmkrnlcom subject electrical wiring faq question 120vac outlet wiring replyto wirefaq@ferretocunixonca keywords 120 240 ac outlets wiring power shock gfci expires 15 may 93 213516 pdt distribution world organization kernel mode systems san diego ca lines 1547 since electrical wiring questions turn time time scielectronics answers always apparent even skilled electronics hijacking following faq posting copy i've asked writers crosspost scielectronics future jeh@cmkrnlcom xnews cmkrnl newsanswers 6685 newsgroups miscconsumershouserecwoodworkingnewsanswersmiscanswersrecanswers subject electrical wiring faq messageid wirefaq733900891@ecicrl clewis@ferretocunixonca chris lewis date 4 apr 93 052149 gmt replyto wirefaq@ferretocunixonca wiring faq commentary reception followupto poster exp

In [109]:
def spectral_cluster(vetor, termos, k=6):
    spectral = SpectralClustering(n_clusters=k, random_state=0, affinity='nearest_neighbors', n_init=10).fit(vetor)
    
    top_docs = []
    top_tokens = []
    for i in range(spectral.n_clusters):
        # Obtém os índices dos top 10 documentos de cada cluster
        cluster_docs = np.where(spectral.labels_ == i)[0]
        cluster_scores = vetor[cluster_docs].sum(axis=1)
        top_docs_idx = np.argsort(cluster_scores, axis=0)[::-1][:10].astype(int)
        top_docs.append(cluster_docs[top_docs_idx])

        # Obtém os índices dos top 20 tokens de cada cluster
        cluster_indices = np.where(spectral.labels_ == i)[0]
        cluster_vectors = vetor[cluster_indices]
        cluster_mean = np.asarray(cluster_vectors.mean(axis=0)).flatten()
        top_tokens_idx = np.argsort(cluster_mean)[::-1][:20]
        cluster_tokens = [termos[j] for j in top_tokens_idx]
        top_tokens.append(cluster_tokens)
    
    # Imprime os top  documentos de cada cluster
    for i, docs in enumerate(top_docs):
        print(f"Cluster {i}:\nTop 20 tokens representativos: \n\t{', '.join(top_tokens[i])}\nTop 10 docs:")
        
        for doc_idx in docs:
            print(f"\t{documentos[doc_idx[0]]}")
        print("="*80 + "\n")

spectral_cluster(tfidf_vectors, tfidf_vectorizer.get_feature_names_out())


Cluster 0:
Top 20 tokens representativos: 
	ground, wire, outlet, wiring, neutral, outlets, breaker, connected, 120vac, question, box, electrical, panel, circuits, circuit, screw, one, current, writes, may
Top 10 docs:
	jeh@cmkrnlcom subject electrical wiring faq question 120vac outlet wiring replyto wirefaq@ferretocunixonca keywords 120 240 ac outlets wiring power shock gfci expires 15 may 93 213516 pdt distribution world organization kernel mode systems san diego ca lines 1547 since electrical wiring questions turn time time scielectronics answers always apparent even skilled electronics hijacking following faq posting copy i've asked writers crosspost scielectronics future jeh@cmkrnlcom xnews cmkrnl newsanswers 6685 newsgroups miscconsumershouserecwoodworkingnewsanswersmiscanswersrecanswers subject electrical wiring faq messageid wirefaq733900891@ecicrl clewis@ferretocunixonca chris lewis date 4 apr 93 052149 gmt replyto wirefaq@ferretocunixonca wiring faq commentary reception follo

Os documentos são classificados com base na probabilidade logarítmica de pertencerem ao cluster, enquanto as palavras são classificadas com base nos valores médios de TF-IDF.

In [8]:
def gaussian_mixture_cluster(vetor, termos, k=4):
    # Classifica por Gaussian Mixture Model (k=4)
    gmm = GaussianMixture(n_components=k, random_state=0).fit(vetor.toarray())

    # Atribui os documentos aos clusters
    labels = gmm.predict(vetor.toarray())

    # Obtém os top 10 documentos e top 20 tokens de cada cluster
    top_documents = []
    top_tokens = []
    for i in range(gmm.n_components):
        cluster_indices = np.where(labels == i)[0]
        cluster_vectors = vetor[cluster_indices]
        
        # Top 10 documentos
        top_docs_idx = np.argsort(gmm.score_samples(vetor.toarray())[cluster_indices])[::-1][:10]
        top_docs = [newsgroups_train.data[j] for j in cluster_indices[top_docs_idx]]
        top_documents.append(top_docs)
        
        # Top 20 tokens
        cluster_mean = np.asarray(cluster_vectors.mean(axis=0)).flatten()
        top_tokens_idx = np.argsort(cluster_mean)[::-1][:20]
        cluster_tokens = [termos[j] for j in top_tokens_idx]
        top_tokens.append(cluster_tokens)

    # Imprime os top 10 documentos e top 20 tokens de cada cluster
    for i, docs in enumerate(top_documents):
        print(f"Cluster {i}:\nTop 20 tokens representativos: \n\t{', '.join(top_tokens[i])}\nTop 10 docs:")
        for doc in docs:
            print("\t"+doc.replace('\n', ''))
        print("="*80 + "\n")

gaussian_mixture_cluster(tfidf_vectors, tfidf_vectorizer.get_feature_names_out())

Cluster 0:
Top 20 tokens representativos: 
	subject, university, thanks, lines, organization, nntppostinghost, anyone, please, know, email, information, advance, would, help, internet, looking, distribution, 11, version, hi
Top 10 docs:
	From: rao@cse.uta.edu (Rao Venkatesh Simha)Subject: xrn, xarchie for HP 9000/730 - ASAPNntp-Posting-Host: cse.uta.eduOrganization: Computer Science Engineering at the University of Texas at ArlingtonLines: 10	Hi,	I need xrn and xarchie for the HP's (9000/730, version 8 OS), either inthe source form or, (preferably) in executable form. Please suggestwhere I can find this, 	Send e-mail to: rao@cse.uta.eduThanks in advance,Rao.-- SSC
	From: rao@cse.uta.edu (Rao Venkatesh Simha)Subject: xrn , xarchie for HP'sNntp-Posting-Host: cse.uta.eduOrganization: Computer Science Engineering at the University of Texas at ArlingtonLines: 10	Hi,	I need xrn and xarchie for the HP's (9000/730, version 8 OS), either inthe source form or, (preferably) in executable form. Pl

Como o Agglomerative Clustering não fornece uma função de pontuação, usamos a soma dos valores TF-IDF para classificar os documentos dentro de cada cluster. Em seguida, selecionamos os top 10 documentos e as top 20 palavras de cada cluster, classificando as palavras com base nos valores médios de TF-IDF.

In [11]:
def agg_cluster(vetor, termos):
    # Classifica por Agglomerative Clustering (k=4)
    agg_clustering = AgglomerativeClustering(n_clusters=4).fit(vetor.toarray())

    # Atribui os documentos aos clusters
    labels = agg_clustering.labels_

    # Obtém os top 10 documentos e top 20 tokens de cada cluster
    top_documents = []
    top_tokens = []
    for i in range(agg_clustering.n_clusters_):
        cluster_indices = np.where(labels == i)[0]
        cluster_vectors = vetor[cluster_indices]
        
        # Top 10 documentos
        # Como o Agglomerative Clustering não fornece uma função de pontuação, usaremos a soma dos valores TF-IDF
        top_docs_idx = np.argsort(np.asarray(cluster_vectors.sum(axis=1)).flatten())[::-1][:10]
        top_docs = [newsgroups_train.data[j] for j in cluster_indices[top_docs_idx]]
        top_documents.append(top_docs)
        
        # Top 20 tokens
        cluster_mean = np.asarray(cluster_vectors.mean(axis=0)).flatten()
        top_tokens_idx = np.argsort(cluster_mean)[::-1][:20]
        cluster_tokens = [termos[j] for j in top_tokens_idx]
        top_tokens.append(cluster_tokens)

    # Imprime os top 10 documentos e top 20 tokens de cada cluster

    for i, docs in enumerate(top_documents):
        print(f"Cluster {i}:\nTop 20 tokens representativos: \n\t{', '.join(top_tokens[i])}\nTop 10 docs:")
        for doc in docs:
            print("\t"+doc.replace('\n', ''))
        print("="*80 + "\n")

agg_cluster(tfidf_vectors, tfidf_vectorizer.get_feature_names_out())

Cluster 0:
Top 20 tokens representativos: 
	lines, subject, organization, would, university, nntppostinghost, one, use, like, thanks, know, anyone, get, window, writes, article, need, server, help, could
Top 10 docs:
	From: dbl@visual.com (David B. Lewis)Subject: comp.windows.x Frequently Asked Questions (FAQ) 4/5Summary: useful information about the X Window SystemArticle-I.D.: visual.C52Ep6.97pExpires: Sun, 2 May 1993 00:00:00 GMTReply-To: faq%craft@uunet.uu.net (X FAQ maintenance address)Organization: VISUAL, Inc.Lines: 968Archive-name: x-faq/part4Last-modified: 1993/04/04----------------------------------------------------------------------Subject:  80)! Where can I get an X-based plotting program?These usually are available from uucp sites such as uunet or other sites asmarked; please consult the archie server to find more recent versions. gnuplot	X (xplot), PostScript and a bunch of other drivers.	export.lcs.mit.edu [and elsewhere]:contrib/gnuplot3.1.tar.Z gl_plot	X output only [

O parâmetro "**eps**" controla a distância máxima entre pontos para que eles sejam considerados parte do mesmo cluster. Se o valor de "eps" for muito alto, muitos pontos podem ser agrupados em um único cluster, enquanto se for muito baixo, muitos clusters pequenos podem ser criados. 

O parâmetro "**min_samples**" controla o número mínimo de pontos que devem estar dentro da distância "eps" para que um cluster seja formado. Se o valor de "min_samples" for muito alto, pode ser difícil formar clusters, enquanto se for muito baixo, muitos clusters pequenos podem ser criados. 

In [12]:
from sklearn.preprocessing import Normalizer

# Extrai as features do dataset usando TF-IDF
vectorizer = tfidf_vectorizer
X = tfidf_vectors

# Normaliza os dados para melhorar o desempenho do DBSCAN
normalizer = Normalizer()
X_normalized = normalizer.fit_transform(X)

# Classifica por DBSCAN
dbscan = DBSCAN(eps=0.95, min_samples=10).fit(X_normalized)

# Atribui os documentos aos clusters
labels = dbscan.labels_

# Obtém os top 10 documentos e top 20 tokens de cada cluster
unique_labels = np.unique(labels)
unique_labels = unique_labels[unique_labels != -1]  # Remove o rótulo de ruído (-1)

if len(unique_labels) == 0:
    print("Nenhum cluster significativo encontrado. Tente ajustar os hiperparâmetros do DBSCAN.")
else:
    print(f'Agrupado em {len(unique_labels)} clusters.')
    top_documents = []
    top_tokens = []
    for i in unique_labels:
        cluster_indices = np.where(labels == i)[0]
        cluster_vectors = X[cluster_indices]

        # Top 10 documentos
        top_docs_idx = np.argsort(np.asarray(cluster_vectors.sum(axis=1)).flatten())[::-1][:10]
        top_docs = [newsgroups_train.data[j] for j in cluster_indices[top_docs_idx]]
        top_documents.append(top_docs)

        # Top 20 tokens
        cluster_mean = np.asarray(cluster_vectors.mean(axis=0)).flatten()
        top_tokens_idx = np.argsort(cluster_mean)[::-1][:20]
        cluster_tokens = [vectorizer.get_feature_names_out()[j] for j in top_tokens_idx]
        top_tokens.append(cluster_tokens)

    # Imprime os top 10 documentos e top 20 tokens de cada cluster
    for i, docs in enumerate(top_documents):
        print(f"Cluster {i}:\nTop 20 tokens representativos: \n\t{', '.join(top_tokens[i])}\nTop 10 docs:")
        for doc in docs:
            print("\t"+doc.replace('\n', ''))
        print("="*80 + "\n")



Agrupado em 4 clusters.
Cluster 0:
Top 20 tokens representativos: 
	radar, detector, detectors, virginia, law, radio, state, receiver, one, operating, rf, cars, local, claim, car, others, detect, yes, antenna, illegal
Top 10 docs:
	From: ragee@vdoe386.vak12ed.edu (Randy Agee)Subject: Radar detector DETECTORS?Organization: Virginia's Public Education Network (Richmond)Lines: 50Here's one I hope some knowledgeable readers will make a comment orcontribution to:In the State of Virginia radar detectors are illegal, period.  Ifyou are caught with one it will be confiscated on the spot and willnot be returned until after you appear in court and pay your fine. The fine for having a radar detector accessible in a motor vehicle(even if it is not on) is $250.00.  Sorry, tourist, ignorance ofthe law is no excuse - they will get you too!It used to be that the only way the law could be enforced was foran officer to actually see the radar detector.  Not any more!  Manylaw enforcement agencies are now

In [15]:

import numpy as np

# Extrai as features do dataset usando TF-IDF
vectorizer = tfidf_vectorizer
X = tfidf_vectors

# Cria o objeto HDBSCAN
clusterer = hdbscan.HDBSCAN(min_cluster_size=12)# Ajusta o modelo aos dados
clusterer.fit(X)

# Obtenha os rótulos de cluster para cada ponto
labels = clusterer.labels_

# Obtenha o número de clusters
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

# Obtém os top 10 documentos e top 20 tokens de cada cluster
unique_labels = np.unique(labels)
unique_labels = unique_labels[unique_labels != -1]  # Remove o rótulo de ruído (-1)

if len(unique_labels) == 0:
    print("Nenhum cluster significativo encontrado. Tente ajustar os hiperparâmetros do HDBSCAN.")
else:
    print(f'Agrupado em {len(unique_labels)} clusters.')
    top_documents = []
    top_tokens = []
    for i in unique_labels:
        cluster_indices = np.where(labels == i)[0]
        cluster_vectors = X[cluster_indices]

        # Top 10 documentos
        top_docs_idx = np.argsort(np.asarray(cluster_vectors.sum(axis=1)).flatten())[::-1][:10]
        top_docs = [newsgroups_train.data[j] for j in cluster_indices[top_docs_idx]]
        top_documents.append(top_docs)

        # Top 20 tokens
        cluster_mean = np.asarray(cluster_vectors.mean(axis=0)).flatten()
        top_tokens_idx = np.argsort(cluster_mean)[::-1][:20]
        cluster_tokens = [vectorizer.get_feature_names_out()[j] for j in top_tokens_idx]
        top_tokens.append(cluster_tokens)

    # Imprime os top 10 documentos e top 20 tokens de cada cluster
    for i, docs in enumerate(top_documents):
        print(f"Cluster {i}:\nTop 20 tokens representativos: \n\t{', '.join(top_tokens[i])}\nTop 10 docs:")
        for doc in docs:
            print("\t"+doc.replace('\n', ''))
        print("="*80 + "\n")


Agrupado em 6 clusters.
Cluster 0:
Top 20 tokens representativos: 
	cooling, nuclear, towers, water, steam, tower, cool, site, uhuraneoucomedu, air, mayhew, wtm, hot, john, power, bill, heat, cold, figured, ever
Top 10 docs:
	From: exuptr@exu.ericsson.se (Patrick Taylor, The Sounding Board)Subject: Re: How to the disks copy protected.Nntp-Posting-Host: 138.85.253.85Organization: Ericsson Network Systems, Inc.X-Disclaimer: This article was posted by a user at Ericsson.              Any opinions expressed are strictly those of the              user and not necessarily those of Ericsson.Lines: 36In article <1993Apr21.131908.29582@uhura.neoucom.edu> wtm@uhura.neoucom.edu (Bill Mayhew) writes:>From: wtm@uhura.neoucom.edu (Bill Mayhew)>Subject: Re: How to the disks copy protected.>Date: Wed, 21 Apr 1993 13:19:08 GMT>Write a good manual to go with the software.  The hassle of>photocopying the manual is offset by simplicity of purchasing>the package for only $15.  Also, consider offering an in

In [111]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans, SpectralClustering, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Carregar o conjunto de dados 20 News Group Dataset
#dataset = fetch_20newsgroups(subset='all', shuffle=True, random_state=42)

# Pré-processamento dos dados
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if token not in stop_words]
    return tokens

processed_documents = documentos

# Treinar o modelo Word2Vec
model = Word2Vec(processed_documents,  window=5, min_count=5, workers=4)

# Vetorizar os documentos usando Word2Vec
vectorized_documents = [np.mean([model.wv[word] for word in doc], axis=0) for doc in processed_documents]

# Realizar a clusterização usando diferentes algoritmos
kmeans = KMeans(n_clusters=4)
kmeans_labels = kmeans.fit_predict(vectorized_documents)

spectral = SpectralClustering(n_clusters=6)
spectral_labels = spectral.fit_predict(vectorized_documents)

gmm = GaussianMixture(n_components=4)
gmm_labels = gmm.fit_predict(vectorized_documents)

agglomerative = AgglomerativeClustering(n_clusters=4)
agglomerative_labels = agglomerative.fit_predict(vectorized_documents)

dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan_labels = dbscan.fit_predict(vectorized_documents)

# Encontrar os top-10 documentos e top-20 tokens para cada cluster
clusters = {
    'K-Means': kmeans_labels,
    'Spectral Clustering': spectral_labels,
    'Gaussian Mixture': gmm_labels,
    'Agglomerative Clustering': agglomerative_labels,
    'DBSCAN': dbscan_labels
}

top_documents = {}
top_tokens = {}

for cluster_name, labels in clusters.items():
    cluster_centers = []
    for cluster_id in np.unique(labels):
        cluster_indices = np.where(labels == cluster_id)[0]
        cluster_vectors = np.array([vectorized_documents[idx] for idx in cluster_indices])
        cluster_center = np.mean(cluster_vectors, axis=0)
        cluster_centers.append(cluster_center)

        # Encontrar os top-10 documentos mais similares ao centro do cluster
        similarity_matrix = cosine_similarity(cluster_vectors, [cluster_center])
        top_document_indices = np.argsort(similarity_matrix.flatten())[::-1][:10]
        top_documents[cluster_id] = [newsgroups_train.data[cluster_indices[idx]] for idx in top_document_indices]

        # Encontrar os top-20 tokens mais frequentes no cluster
        cluster_tokens = [token for idx in cluster_indices for token in processed_documents[idx]]
        token_frequencies = nltk.FreqDist(cluster_tokens)
        top_tokens[cluster_id] = [token for token, _ in token_frequencies.most_common(20)]
        #print(token_frequencies)

    print(f'Cluster results for {cluster_name}:')
    for cluster_id, center in enumerate(cluster_centers):
        print(f'Cluster {cluster_id}:')
        print('Top Documents:')
        for doc in top_documents[cluster_id]:
            print("\t"+doc.replace('\n', ' '))
        #print('Top Tokens:')
        #for token in top_tokens[cluster_id]:
        #    print(token)
        print('-' * 50)
    print('=' * 80)


Cluster results for K-Means:
Cluster 0:
Top Documents:
	From: atae@spva.ph.ic.ac.uk (Ata Etemadi) Subject: Ideal Operating System (Was: DEATH BLOW TO UNIX) Nntp-Posting-Host: prawn.sp.ph Organization: Imperial College of Science, Technology, and Medicine, London, England Lines: 45  G'Day  Windows NT is a step forward, but not by much. I guess for DOS users who think multi-tasking is really _neat_ (add puke here) and are overjoyed to  be able to use more than 64K of memory, its a major improvement. For Windows users its more like an upgrade with facilities most Unix users take for  granted. Most of  Windows users only use PCs for text processing or running  3rd-party applications anyway so the operating system features are not even  a real issue. My ideal operating system:  	Binary compatible across all plaforms. It should recognize 	binaries and configure on the fly, so I don't have to have 	bin/Mac bin/DOS bin/Dec bin/Sun bin/HP etc...  	Multi-CPU. So I can use the CPUs on my PC and W

# Daqui pra baixo são testes de otimização para generalizar o código


In [110]:
spectral_cluster(vectorized_documents, tfidf_vectorizer.get_feature_names_out())

TypeError: only integer scalar arrays can be converted to a scalar index

In [83]:
# Vetorizar os documentos usando Word2Vec
vectorized_documents = [np.mean([model.wv[word] for word in doc], axis=0) for doc in processed_documents]

# Realizar a clusterização usando diferentes algoritmos
kmeans_labels = KMeans(n_clusters=4).fit_predict(vectorized_documents)
kmeans.predict

<bound method _BaseKMeans.predict of KMeans(n_clusters=4)>

TypeError: object of type 'numpy.int32' has no len()